Первый этап - функционал болталки

In [8]:
#Загрузка библиотек
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

Болталка будет смесью встроенного функционала dialogflow и собственноручного поиска вопроса и ответа из prepared_answers.txt и Otvety.txt

In [3]:
#Предобработка файла с ответами 

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("Otvety.txt", "r", errors="ignore") as fin:
        for line in tqdm_notebook(fin):
            
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [5]:
#Предобработка текста
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [5]:
#Предобработка для модели

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r", errors="ignore") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

In [8]:
# Обучим модель на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
model.save("w2v_model")

In [9]:
#Через библиотеку annoy проходим по всем ответам и вычисляем усредненый вектор, результаты сохраняем
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
            
        counter += 1

index.build(10)
index.save('speaker.ann')

True

In [10]:
#Сохраним список ответов
pickle.dump(index_map, open("index_map",'wb'))

In [11]:
#Загрузим резульаты модели
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')

True

Второй этап - продуктовая часть

In [3]:
index_map = pickle.load(open("index_map",'rb'))

In [11]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

In [25]:
#Загрузим датасет с товаркой
shop_data = pd.read_csv("ProductsDataset.csv")

In [29]:
# Релизуем предобработку текста для классификатора
import re
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("nan", "", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

In [30]:
# Объединим в один стобец название с описанием продукта и предобработаем
shop_data['text']= shop_data['title'].apply(preprocess_text) + ' ' + shop_data['descrirption'].apply(preprocess_text)
# Целевая переменная для классификации
shop_data['target'] = True

In [31]:
# Дополним датасет данными
data = pd.DataFrame(list(index_map.items()),
                   columns=['index', 'text'])
data = data[data.index < len(shop_data)]
data['target'] = False
data = data.drop(['index'], axis=1)
data['text'] = data['text'].apply(preprocess_text)
data_class = pd.concat([shop_data, data], ignore_index = True)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model, metrics, model_selection, preprocessing

In [32]:
# делим выборку
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(data_class['text'], data_class['target'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [33]:
# Векторизируем текст при помощи TfidfVectorizer
tfidf_vec = TfidfVectorizer().fit(train_x.values)
xtrain_tfidf = vectorizer.fit_transform(train_x)
xvalid_tfidf = vectorizer.transform(valid_x)

In [34]:
# Обучим классификатор 
classifier = linear_model.LogisticRegression()
classifier.fit(xtrain_tfidf, train_y)
predictions = classifier.predict(xvalid_tfidf)
accuracy = metrics.accuracy_score(predictions, valid_y)
print("LR, WordLevel TF-IDF: ", accuracy)

LR, WordLevel TF-IDF:  0.9935298750984585


In [35]:
# Сохраняем модель классификатор
pickle.dump(classifier, open("text_classifier",'wb'))
# Загружаем модель классификатор
classifier_model = pickle.load(open("text_classifier", 'rb'))

In [36]:
# Создадим список слов из названия и описания продуктов
titles = []

for line in tqdm_notebook(data.text):
    spls = preprocess_txt(line)
    titles.append(spls)

# Векторизируем список слов при помощи модели Word2Vec и сохраним модель 
titles = [i for i in titles if len(i) > 2]
product_model = Word2Vec(sentences=titles, vector_size=100, min_count=1, window=5)
product_model.save("w2v_product_model")

In [39]:
# Сложим в индексы используем библиотеку annoy. 
product_index = annoy.AnnoyIndex(100 ,'angular')

index_map_titles = {}
counter = 0

for quest in shop_data.title:
    n_w2v = 0
    index_map_titles[counter] = str(shop_data.product_id[counter]) + ' ' + shop_data.title[counter]
    question = preprocess_txt(quest)
    vector = np.zeros(100)
    for word in question:
        if word in product_model.wv:
            vector += product_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    product_index.add_item(counter, vector)
        
    counter += 1

product_index.build(10)
product_index.save('shopx.ann')

True

In [40]:
# Функция для поиска продуктов
def find_product(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in product_model.wv:
            vector += product_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    query_index = product_index.get_nns_by_vector(vector, 1)
    return index_map_titles[query_index[0]]

In [41]:
# Для болталки возьмем обученную ранее модель и созданные индексы
answer_model = Word2Vec.load("w2v_model")
answer_index = annoy.AnnoyIndex(100 ,'angular')
answer_index.load('speaker.ann')

True

In [42]:
# Функция для ответов на запросы "болталки"
def find_answer(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in answer_model.wv:
            vector += answer_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    query_index = answer_index.get_nns_by_vector(vector, 1)
    return index_map[query_index[0]]

In [44]:
# Функция для ответа на запрос пользователя
def get_answer(answer):
    ans = pd.Series([answer], dtype="string")
    answer_tfidf = tfidf_vec.transform(ans) # Векторизируем запрос при помощи TfidfVectorizer
    if classifier.predict(answer_tfidf)==1: # Если запрос был продуктовым, возвращаем id и название продукта
        return find_product(answer)
    else:
        return find_answer(answer) # иначе отправляем запрос в болталку

In [45]:
#Подключение бота
updater = Updater(token='1058946299:AAFn0ESsBoqdzscvF-tYo0wcZvAUS_iJMCw') # Токен API к Telegram
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'bot-nlp-igor-pemi-ff9190fe2c7d.json'# скачнный JSON


DIALOGFLOW_PROJECT_ID = 'bot-nlp-igor-pemi' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'Bot_nlp_igor_bot'  # ID бота из телеграма

In [47]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Добрый день')

def textMessage(bot, update):
    
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.2:
        print(distances[0])
        bot.send_message(chat_id=update.message.chat_id, text="Повторите вопрос")
        return
    bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])
    
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
# Добавляем хендлеры в диспетчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота, если были нажаты Ctrl + C
updater.idle()